In [8]:
import pandas as pd

#### 1. Creating dataframe 
Converting cvs data to pandas dataframe

In [9]:
df = pd.read_csv('dataset.csv')

#### 2. Droping null values
The current data contains null values which can create faulty results, so first step will be to drop such values by using the following command

In [15]:
df.dropna(inplace=True)

In [10]:
df

,BUILDINGKEY,FLOORKEY,FLOORNAME,NURSEKEY,NURSENAME,ROOMKEY,ROOMNAME,BEDKEY,BEDNAME,BEDSTATUS
0,5.0,9.0,First Floor NT,00:00.0,Daycare Surgery,33.0,Daycare NT (Day Care),34.0,101-1,Occupied Discharge
1,5.0,9.0,First Floor NT,00:00.0,Daycare Surgery,33.0,Daycare NT (Day Care),35.0,101-1P,Occupied Discharge
2,5.0,9.0,First Floor NT,00:00.0,Daycare Surgery,33.0,Daycare NT (Day Care),36.0,101-2,Vacant Dirty
3,5.0,9.0,First Floor NT,00:00.0,Daycare Surgery,33.0,Daycare NT (Day Care),37.0,101-2P,Vacant Dirty
4,5.0,9.0,First Floor NT,00:00.0,Daycare Surgery,33.0,Daycare NT (Day Care),38.0,101-3,Vacant Dirty
...,...,...,...,...,...,...,...,...,...,...
1095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 3. Removing duplicates
There are also duplicate values in our given data set so, lets go ahead and remove the repeating data. Hence, reducing the redundent data

In [7]:
filter_duplicate = df.drop_duplicates()

### Data Query 

#### Note
In the given data there's nursename and nursekey so, we have to use one as primary key to get bed data in each ward. There can be two ways to approach this problem:
- Method 1 - We can use nursename as key cause in dataset two nursekey have same nursename, hence there's in redundency.
- Method 2 - We can combine nursename and nursekey to form a unique key to query through the data.

For sake of simplicity of this project let's go with method 1, the similar logic follows for bedname and bedkey

#### 1. Get the count of bed in each ward
We first remove duplicate entries and count beds. Taking into consideration that a single bed can only be present in a single ward, we remove the duplicates by grouping by nursename and finally we count the bednames

In [16]:
filter_duplicate[['NURSENAME','BEDNAME']].drop_duplicates().groupby('NURSENAME',as_index=False).count()

,NURSENAME,BEDNAME
0,Daycare Endoscopy,32
1,Daycare Haemato & Onco Adult,35
2,Daycare Haemato & Onco Paed,11
3,Daycare Imaging,10
4,Daycare Surgery,25
5,NT Admission Lounge,18
6,NT ICU,8
7,NT Labour Room,21
8,NT NICU,4
9,NT Nursery,59


#### 2. Get the count of bed in each room
This follows the similar procedure as first qurey, hence we again group data but this time by roomname an count the beds

In [20]:
filter_duplicate[['ROOMNAME','BEDNAME']].drop_duplicates().groupby('ROOMNAME',as_index=False).count()

,ROOMNAME,BEDNAME
0,CCU Room 201 ST (ICU/CCU),11
1,Daycare NT (Day Care),25
2,Haematology & Oncology Daycare Adult (Day Care...,35
3,Haematology & Oncology Daycare Paeds (Day Care...,11
4,ICU NT (ICU/CCU),8
...,...,...
270,Room 585 (Double Bed),2
271,Room 586 (Double Bed),2
272,Room 587 (Double Bed),2
273,Room 588 (Four Bedded),2


#### 3. Get the count of the bed under each status

In [21]:
filter_duplicate[['BEDSTATUS','BEDNAME']].drop_duplicates().groupby('BEDSTATUS',as_index=False).count()

,BEDSTATUS,BEDNAME
0,Available,337
1,Blocked,1
2,Occupied Discharge,42
3,Occupied(Female),73
4,Occupied(Male),73
5,Vacant Dirty,159


#### 4. Calculate the overall occupancy percentage(total occupied/total bed)
For this we can simply add all the occupied discharge, occupied(female), occupied(male) bed to get the count of occupied bed.

In [27]:
bed_count_by_status = filter_duplicate[['BEDSTATUS','BEDNAME']].drop_duplicates().groupby('BEDSTATUS',as_index=False).count()

In [32]:
total_occupied_bed = bed_count_by_status[bed_count_by_status['BEDSTATUS']=='Occupied Discharge']['BEDNAME'].sum()+bed_count_by_status[bed_count_by_status['BEDSTATUS']=='Occupied(Female)']['BEDNAME'].sum()+bed_count_by_status[bed_count_by_status['BEDSTATUS']=='Occupied(Male)']['BEDNAME'].sum()

In [29]:
total_bed = bed_count_by_status['BEDNAME'].sum()

In [34]:
percentage = (total_occupied_bed/total_bed)*100

In [36]:
percentage

27.44525547445256

#### 5. Calculate occupancy for each ward
We first calculate the the number of occupied beds in each ward then divide the beds occupied by total beds in each ward to calculate the occupancy of each ward 

In [43]:
filter_ward = filter_duplicate[['NURSENAME','BEDNAME','BEDSTATUS']].drop_duplicates()

In [44]:
arr = ['Occupied(Male)','Occupied(Female)','Occupied Discharge']
ward_occupied_bed = filter_ward[filter_ward['BEDSTATUS'].isin(arr)]

In [45]:
x = ward_occupied_bed.groupby('NURSENAME')['BEDSTATUS'].count()

In [46]:
y = filter_ward.groupby('NURSENAME')['BEDSTATUS'].count()

In [48]:
x/y

NURSENAME
Daycare Endoscopy               0.187500
Daycare Haemato & Onco Adult    0.057143
Daycare Haemato & Onco Paed          NaN
Daycare Imaging                      NaN
Daycare Surgery                 0.080000
NT Admission Lounge                  NaN
NT ICU                          0.625000
NT Labour Room                       NaN
NT NICU                         0.500000
NT Nursery                      0.016949
NT Paed HDU                          NaN
NT Ward 2D                      0.290909
NT Ward 3D                      0.538462
NT Ward 4D                      0.157895
NT Ward 5C                           NaN
NT Ward 5D                      0.615385
ST CCU                          0.636364
ST HDU                          0.666667
ST ICU                          0.555556
ST Ward 2A                      0.566667
ST Ward 3A                      0.870968
ST Ward 3B                      0.111111
ST Ward 4A                      0.266667
ST Ward 4B                      0.090909
ST War

####  6. Calculate occupancy for each room
The proceduer will be same as problem 6

In [50]:
filter_room = filter_duplicate[['ROOMNAME','BEDNAME','BEDSTATUS']].drop_duplicates()

In [52]:
room_occupied_bed = filter_room[filter_room['BEDSTATUS'].isin(arr)]

In [53]:
x1 = room_occupied_bed.groupby('ROOMNAME')['BEDSTATUS'].count()

In [54]:
y1 = filter_room.groupby('ROOMNAME')['BEDSTATUS'].count()

In [59]:
x1/y1

ROOMNAME
CCU Room 201 ST (ICU/CCU)                                          0.636364
Daycare NT (Day Care)                                              0.080000
Haematology & Oncology Daycare Adult (Day Care Chemo)              0.057143
Haematology & Oncology Daycare Paeds (Day Care Chemo (Private))         NaN
ICU NT  (ICU/CCU)                                                  0.625000
                                                                     ...   
Room 585 (Double Bed)                                              0.500000
Room 586 (Double Bed)                                              0.500000
Room 587 (Double Bed)                                              0.500000
Room 588 (Four Bedded)                                                  NaN
Room G03 (Day Care)                                                     NaN
Name: BEDSTATUS, Length: 275, dtype: float64

#### 7. Calculate count of bed with all the different bed statuses for each ward
We get the data and group by nursename and bedstatus and get the count for bedname

In [61]:
filter_duplicate[['NURSENAME','BEDNAME','BEDSTATUS']].drop_duplicates().groupby(['NURSENAME','BEDSTATUS'],as_index=False).count()

,NURSENAME,BEDSTATUS,BEDNAME
0,Daycare Endoscopy,Available,16
1,Daycare Endoscopy,Occupied Discharge,6
2,Daycare Endoscopy,Vacant Dirty,10
3,Daycare Haemato & Onco Adult,Available,3
4,Daycare Haemato & Onco Adult,Occupied Discharge,2
...,...,...,...
83,ST Ward 5A,Vacant Dirty,7
84,ST Ward 5B,Available,8
85,ST Ward 5B,Occupied(Female),4
86,ST Ward 5B,Occupied(Male),7


#### 7. Calculate count of bed with all the different bed statuses for each room
We get the data and group by roomname and bedstatus and get the count for bedname

In [62]:
filter_duplicate[['ROOMNAME','BEDNAME','BEDSTATUS']].drop_duplicates().groupby(['ROOMNAME','BEDSTATUS'],as_index=False).count()

,ROOMNAME,BEDSTATUS,BEDNAME
0,CCU Room 201 ST (ICU/CCU),Available,4
1,CCU Room 201 ST (ICU/CCU),Occupied(Female),1
2,CCU Room 201 ST (ICU/CCU),Occupied(Male),6
3,Daycare NT (Day Care),Available,6
4,Daycare NT (Day Care),Occupied Discharge,2
...,...,...,...
365,Room 587 (Double Bed),Occupied(Female),1
366,Room 587 (Double Bed),Vacant Dirty,1
367,Room 588 (Four Bedded),Available,2
368,Room G03 (Day Care),Available,4
